# 10.7. Sequence-to-Sequence Learning for Machine Translation

#  Sequence-to-Sequence Learning for Machine Translation


In so-called sequence-to-sequence problems such as machine translation
(as discussed in Section 10.5),
where inputs and outputs each consist 
of variable-length unaligned sequences,
we generally rely on encoder--decoder architectures
(Section 10.6).
In this section,
we will demonstrate the application 
of an encoder--decoder architecture,
where both the encoder and decoder 
are implemented as RNNs,
to the task of machine translation
(Cho et al., 2014, Sutskever et al., 2014).

Here, the encoder RNN will take a variable-length sequence as input 
and transform it into a fixed-shape hidden state.
Later, in Section 11,
we will introduce attention mechanisms, 
which allow us to access encoded inputs
without having to compress the entire input
into a single fixed-length representation.

Then to generate the output sequence, 
one token at a time,
the decoder model, 
consisting of a separate RNN,
will predict each successive target token
given both the input sequence
and the preceding tokens in the output.
During training, the decoder will typically
be conditioned upon the preceding tokens
in the official "ground truth" label. 
However, at test time, we will want to condition
each output of the decoder on the tokens already predicted. 
Note that if we ignore the encoder,
the decoder in a sequence-to-sequence architecture 
behaves just like a normal language model.
Fig. 10.7.1 illustrates
how to use two RNNs
for sequence-to-sequence learning
in machine translation.


![Sequence-to-sequence learning with an RNN encoder and an RNN decoder.](http://d2l.ai/_images/seq2seq.svg)
Fig. 10.7.1 Sequence-to-sequence learning with an RNN encoder and an RNN decoder.

In Fig. 10.7.1,
the special "<eos>" token
marks the end of the sequence.
Our model can stop making predictions
once this token is generated.
At the initial time step of the RNN decoder,
there are two special design decisions to be aware of:
First, we begin every input with a special 
beginning-of-sequence "<bos>" token.
Second, we may feed
the final hidden state of the encoder
into the decoder
at every single decoding time step (Cho et al., 2014).
In some other designs,
such as that of Sutskever et al. (2014),
the final hidden state of the RNN encoder
is used
to initiate the hidden state of the decoder
only at the first decoding step.

In [10]:
use strict;
use warnings;
use Data::Dump qw(dump);
use d2l;
IPerl->load_plugin('Chart::Plotly');

# 10.7.1. Teacher Forcing

While running the encoder on the input sequence is relatively straightforward, handling the input and output of the decoder requires more care. The most common approach is sometimes called teacher forcing. Here, the original target sequence (token labels) is fed into the decoder as input. More concretely, the special beginning-of-sequence token and the original target sequence, excluding the final token, are concatenated as input to the decoder, while the decoder output (labels for training) is the original target sequence, shifted by one token: “<bos>”, “Ils”, “regardent”, “.” ->
 “Ils”, “regardent”, “.”, “<eos>” (Fig. 10.7.1).

Our implementation in Section 10.5.3 prepared training data for teacher forcing, where shifting tokens for self-supervised learning is similar to the training of language models in Section 9.3. An alternative approach is to feed the predicted token from the previous time step as the current input to the decoder.

In the following, we explain the design depicted in Fig. 10.7.1 in greater detail. We will train this model for machine translation on the English–French dataset as introduced in Section 10.5.

# 10.7.2. Encoder

## Encoder

Recall that the encoder transforms an input sequence of variable length
into a fixed-shape *context variable* $\mathbf{c}$ (see Fig. 10.7.1).


Consider a single sequence example (batch size 1).
Suppose the input sequence is $x_1, \ldots, x_T$, 
such that $x_t$ is the $t^{\textrm{th}}$ token.
At time step $t$, the RNN transforms
the input feature vector $\mathbf{x}_t$ for $x_t$
and the hidden state $\mathbf{h} _{t-1}$ 
from the previous time step 
into the current hidden state $\mathbf{h}_t$.
We can use a function $f$ to express 
the transformation of the RNN's recurrent layer:

$$\mathbf{h}_t = f(\mathbf{x}_t, \mathbf{h}_{t-1}). $$

In general, the encoder transforms 
the hidden states at all time steps
into a context variable through a customized function $q$:

$$\mathbf{c} =  q(\mathbf{h}_1, \ldots, \mathbf{h}_T).$$

For example, inFig. 10.7.1,
the context variable is just the hidden state $\mathbf{h}_T$
corresponding to the encoder RNN's representation
after processing the final token of the input sequence.

In this example, we have used a unidirectional RNN
to design the encoder,
where the hidden state only depends on the input subsequence 
at and before the time step of the hidden state.
We can also construct encoders using bidirectional RNNs.
In this case, a hidden state depends on the subsequence before and after the time step 
(including the input at the current time step), 
which encodes the information of the entire sequence.


Now let's [**implement the RNN encoder**].
Note that we use an *embedding layer*
to obtain the feature vector for each token in the input sequence.
The weight of an embedding layer is a matrix,
where the number of rows corresponds to 
the size of the input vocabulary (`vocab_size`)
and number of columns corresponds to 
the feature vector's dimension (`embed_size`).
For any input token index $i$,
the embedding layer fetches the $i^{\textrm{th}}$ row 
(starting from 0) of the weight matrix
to return its feature vector.
Here we implement the encoder with a multilayer GRU.

In [ ]:
def init_seq2seq(module):  #@save
    """Initialize weights for sequence-to-sequence learning."""
    if type(module) == nn.Linear:
         nn.init.xavier_uniform_(module.weight)
    if type(module) == nn.GRU:
        for param in module._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(module._parameters[param])

class Seq2SeqEncoder(d2l.Encoder):  #@save
    """The RNN encoder for sequence-to-sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = d2l.GRU(embed_size, num_hiddens, num_layers, dropout)
        self.apply(init_seq2seq)

    def forward(self, X, *args):
        # X shape: (batch_size, num_steps)
        embs = self.embedding(X.t().type(torch.int64))
        # embs shape: (num_steps, batch_size, embed_size)
        outputs, state = self.rnn(embs)
        # outputs shape: (num_steps, batch_size, num_hiddens)
        # state shape: (num_layers, batch_size, num_hiddens)
        return outputs, state

In [54]:
package Seq2SeqEncoder{
use base ("d2l::Encoder");
sub new {
    my ($class, %args) = (shift, d2l->get_arguments(vocab_size=> undef,
                                                    embed_size=> undef ,
                                                    num_hiddens=> undef,
                                                    num_layers=> undef,
                                                    dropout => 0, \@_));
    my  $self = $class->SUPER::new(); 
    $self->{embedding} = mx->gluon->nn->Embedding($args{vocab_size}, $args{embed_size}); 
    $self->{rnn} =  new d2l::GRU($args{num_hiddens}, $args{num_layers}, $args{dropout});
    map {$self->register_child($self->{$_})} ('embedding','rnn');
    
    $self->initialize(mx->init->Xavier());
    return bless ($self, $class);
}

sub forward{
    my ($self , $X, %args) = (splice(@_, 0, 2), d2l->get_arguments(\@_));
        # X shape: (batch_size, num_steps)
        # embs = self.embedding(d2l.transpose(X))
        my $embs = $self->{embedding}->forward(mx->nd->transpose($X));
        # embs shape: (num_steps, batch_size, embed_size)
        my ($outputs, $state)= @{$self->{rnn}->forward($embs)};
        # outputs shape: (num_steps, batch_size, num_hiddens)
        # state shape: (num_layers, batch_size, num_hiddens)
    return [$outputs, $state];
}
1;
}


1

Warning: Subroutine new redefined at reply input line 3.

Subroutine forward redefined at reply input line 18.


Let's use a concrete example
to illustrate the above encoder implementation.
Below, we instantiate a two-layer GRU encoder
whose number of hidden units is 16.
Given a minibatch of sequence inputs `X`
(batch size $=4$; number of time steps $=9$),
the hidden states of the final layer
at all the time steps
(`enc_outputs` returned by the encoder's recurrent layers)
are a tensor of shape
(number of time steps, batch size, number of hidden units).

In [ ]:
vocab_size, embed_size, num_hiddens, num_layers = 10, 8, 16, 2
batch_size, num_steps = 4, 9
encoder = Seq2SeqEncoder(vocab_size, embed_size, num_hiddens, num_layers)
X = torch.zeros((batch_size, num_steps))
enc_outputs, enc_state = encoder(X)
d2l.check_shape(enc_outputs, (num_steps, batch_size, num_hiddens))

In [67]:
my ($vocab_size, $embed_size, $num_hiddens, $num_layers) = (10, 8, 16, 2);
my ($batch_size, $num_steps) = (4, 9);
my $encoder = new Seq2SeqEncoder ($vocab_size, $embed_size, $num_hiddens, $num_layers); 
print $encoder, "\n";
print $encoder->{embedding}, "\n";
print $encoder->{rnn}{rnn},"\n";
my $X = mx->nd->zeros([$batch_size, $num_steps]);
my ($enc_outputs, $enc_state) = @{$encoder->forward($X)};
print $enc_outputs, $enc_state, "\n";
d2l->check_shape ($enc_outputs, [$num_steps, $batch_size, $num_hiddens]);
d2l->list_params ($encoder);

Seq2SeqEncoder(

)
Embedding(10 -> 8, float32)
GRU(16, TNC, num_layers=2)
<AI::MXNet::NDArray 9x4x16 @cpu(0)><AI::MXNet::NDArray 2x4x16 @cpu(0)>
dense:
	0	dense1_weight	Parameter dense1_weight (shape=(10, 0), dtype=float32, stype=default)
	1	dense1_bias	Parameter dense1_bias (shape=(10), dtype=float32, stype=default)
rnn:
	0	gru5_l0_i2h_weight	<AI::MXNet::NDArray 48x8 @cpu(0)>
	1	gru5_l0_h2h_weight	<AI::MXNet::NDArray 48x16 @cpu(0)>
	2	gru5_l0_i2h_bias	<AI::MXNet::NDArray 48 @cpu(0)>
	3	gru5_l0_h2h_bias	<AI::MXNet::NDArray 48 @cpu(0)>
	4	gru5_l1_i2h_weight	<AI::MXNet::NDArray 48x16 @cpu(0)>
	5	gru5_l1_h2h_weight	<AI::MXNet::NDArray 48x16 @cpu(0)>
	6	gru5_l1_i2h_bias	<AI::MXNet::NDArray 48 @cpu(0)>
	7	gru5_l1_h2h_bias	<AI::MXNet::NDArray 48 @cpu(0)>
embedding:
	0	embedding5_weight	<AI::MXNet::NDArray 10x8 @cpu(0)>


Since we are using a GRU here, the shape of the multilayer hidden states at the final time step is (number of hidden layers, batch size, number of hidden units).

In [ ]:
d2l.check_shape(enc_state, (num_layers, batch_size, num_hiddens))

In [38]:
d2l->check_shape($enc_state, [$num_layers, $batch_size, $num_hiddens])

1

# 10.7.3. Decoder

Given a target output sequence $y_1, y_2, \ldots, y_{T'}$
for each time step $t'$
(we use $t^\prime$ to differentiate from the input sequence time steps),
the decoder assigns a predicted probability
to each possible token occurring at step $y_{t'+1}$
conditioned upon the previous tokens in the target
$y_1, \ldots, y_{t'}$ 
and the context variable 
$\mathbf{c}$, i.e., $P(y_{t'+1} \mid y_1, \ldots, y_{t'}, \mathbf{c})$.

To predict the subsequent token $t^\prime+1$ in the target sequence,
the RNN decoder takes the previous step's target token $y_{t^\prime}$,
the hidden RNN state from the previous time step $\mathbf{s}_{t^\prime-1}$,
and the context variable $\mathbf{c}$ as its input,
and transforms them into the hidden state 
$\mathbf{s}_{t^\prime}$ at the current time step.
We can use a function $g$ to express 
the transformation of the decoder's hidden layer:

$$\mathbf{s}_{t^\prime} = g(y_{t^\prime-1}, \mathbf{c}, \mathbf{s}_{t^\prime-1}).$$
(10.7.3)

After obtaining the hidden state of the decoder,
we can use an output layer and the softmax operation 
to compute the predictive distribution
$p(y_{t^{\prime}+1} \mid y_1, \ldots, y_{t^\prime}, \mathbf{c})$ 
over the subsequent output token ${t^\prime+1}$.

Following Fig. 10.7.1,
when implementing the decoder as follows,
we directly use the hidden state at the final time step
of the encoder
to initialize the hidden state of the decoder.
This requires that the RNN encoder and the RNN decoder 
have the same number of layers and hidden units.
To further incorporate the encoded input sequence information,
the context variable is concatenated
with the decoder input at all the time steps.
To predict the probability distribution of the output token,
we use a fully connected layer
to transform the hidden state 
at the final layer of the RNN decoder.

In [ ]:
class Seq2SeqDecoder(d2l.Decoder):
    """The RNN decoder for sequence to sequence learning."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = d2l.GRU(embed_size+num_hiddens, num_hiddens,
                           num_layers, dropout)
        self.dense = nn.LazyLinear(vocab_size)
        self.apply(init_seq2seq)

    def init_state(self, enc_all_outputs, *args):
        return enc_all_outputs

    def forward(self, X, state):
        # X shape: (batch_size, num_steps)
        # embs shape: (num_steps, batch_size, embed_size)
        embs = self.embedding(X.t().type(torch.int32))
        enc_output, hidden_state = state
        # context shape: (batch_size, num_hiddens)
        context = enc_output[-1]
        # Broadcast context to (num_steps, batch_size, num_hiddens)
        context = context.repeat(embs.shape[0], 1, 1)
        # Concat at the feature dimension
        embs_and_context = torch.cat((embs, context), -1)
        outputs, hidden_state = self.rnn(embs_and_context, hidden_state)
        outputs = self.dense(outputs).swapaxes(0, 1)
        # outputs shape: (batch_size, num_steps, vocab_size)
        # hidden_state shape: (num_layers, batch_size, num_hiddens)
        return outputs, [enc_output, hidden_state]

In [105]:
package Seq2SeqDecoder {
    use base ("d2l::Decoder");

    sub new {
        my ($class, %args) = (shift, d2l->get_arguments(vocab_size => undef,
                                                        embed_size => undef,
                                                        num_hiddens => undef,
                                                        num_layers => undef,
                                                        dropout => 0, @_));
        my $self = $class->SUPER::new();
        $self->{embedding} = mx->gluon->nn->Embedding($args{vocab_size}, $args{embed_size});
        $self->{rnn} = d2l::GRU->new($args{embed_size} + $args{num_hiddens}, $args{num_hiddens}, $args{num_layers}, $args{dropout});
        $self->{dense} = mx->gluon->nn->LazyLinear($args{vocab_size});
        map {$self->register_child($self->{$_})} ('embedding', 'rnn', 'dense');
        $self->{dense}= mx->gluon->nn->Dense($args{vocab_size}, flatten=>0);
        $self->initialize(mx->init->init_seq2seq());
        return bless ($self, $class);
    }

      sub init_state {
        my ($self, $enc_all_outputs, @args) = @_;
        my %args = d2l->get_arguments(@args);
        return $enc_all_outputs;
    }

    sub forward {
        #my ($self, $X, $state) = (splice(@_, 0, 2), d2l->get_arguments(\@_));
        my ($self, $X, $state) = (splice(@_, 0, 3));
        my %args = d2l->get_arguments(\@_);
        my $embs = $self->{embedding}->forward(mx->nd->transpose($X));
        my ($enc_output, $hidden_state) = @{$state};
        my $context = $enc_output->slice([0, $enc_output->shape->[-1] - 1])->squeeze();
        $context = mx->nd->repeat($context, $embs->shape->[0])->expand_dims(0);
        my $embs_and_context = mx->nd->concatenate([$embs, $context], axis => -1);
        my ($outputs, $new_hidden_state) = @{$self->{rnn}->forward($embs_and_context, $hidden_state)};
        $outputs = $self->{dense}->forward($outputs)->swapaxes(0, 1);
        return [$outputs, [$enc_output, $new_hidden_state]];
    }

    1;
}


1

Warning: Subroutine new redefined at reply input line 4.

Subroutine init_state redefined at reply input line 20.

Subroutine forward redefined at reply input line 26.


To illustrate the implemented decoder, below we instantiate it with the same hyperparameters from the aforementioned encoder. As we can see, the output shape of the decoder becomes (batch size, number of time steps, vocabulary size), where the final dimension of the tensor stores the predicted token distribution.

In [ ]:
decoder = Seq2SeqDecoder(vocab_size, embed_size, num_hiddens, num_layers)
state = decoder.init_state(encoder(X))
dec_outputs, state = decoder(X, state)
d2l.check_shape(dec_outputs, (batch_size, num_steps, vocab_size))
d2l.check_shape(state[1], (num_layers, batch_size, num_hiddens))

In [106]:
my $decoder = new Seq2SeqDecoder($vocab_size, $embed_size, $num_hiddens, $num_layers); 
print $decoder, "\n";
print $decoder->{embedding}, "\n";
print $decoder->{rnn}{rnn}, "\n"; 
print $decoder->{dense}, "\n";
my $state = $decoder->init_state ($encoder->forward ($X)); 
print $state, "\n";
print scalar @$state, "\n";
print "state->[0]: ", $state->[0], "\n";
print "state->[1]: ", $state->[1], "\n";
(my $dec_outputs, $state) = @{$decoder->forward ($X, $state)};
print "state->[1]: ", $state->[1], "\n";
d2l->check_shape ($dec_outputs, [$batch_size, $num_steps, $vocab_size]);
d2l->check_shape ($state->[1], [$num_layers, $batch_size, $num_hiddens]); 
d2l->list_params ($decoder);

Warning: Odd number of elements in hash assignment at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/d2l.pm line 2673.


Error: Can't use string ("2") as an ARRAY ref while "strict refs" in use at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/d2l.pm line 2676.


The layers in the above RNN encoder--decoder model 
are summarized in Fig. 10.7.2.

![Layers in an RNN encoder--decoder model.](../img/seq2seq-details.svg)

Fig. 10.7.2 Layers in a RNN encoder-decoder model.

# 10.7.4. Encoder–Decoder for Sequence-to-Sequence Learning
Putting it all together in code yields the following:

In [ ]:
class Seq2Seq(d2l.EncoderDecoder):  #@save
    """The RNN encoder--decoder for sequence to sequence learning."""
    def __init__(self, encoder, decoder, tgt_pad, lr):
        super().__init__(encoder, decoder)
        self.save_hyperparameters()

    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)

    def configure_optimizers(self):
        # Adam optimizer is used here
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [17]:
package Seq2Seq {
    use base ("d2l::EncoderDecoder");

    sub new {
        my ($class, $encoder, $decoder, $tgt_pad, $lr) = @_;
        my $self = $class->SUPER::new($encoder, $decoder);
        $self->{tgt_pad} = $tgt_pad;
        $self->{lr} = $lr;
        $self->save_hyperparameters();
        return bless ($self, $class);
    }

    sub validation_step {
        my ($self, $batch) = @_;
        my $Y_hat = $self->forward(@$batch[0 .. $#{$batch} - 2]);
        $self->plot('loss', $self->loss($Y_hat, $batch->[-1]), train => 0);
    }

    sub configure_optimizers {
        my ($self) = @_;
        # Assuming Adam optimizer is used here
        my $adam = mx->optimizer->Adam(learning_rate => $self->{lr});
        return $adam;
    }

    1;
}


1

# 10.7.5. Loss Function with Masking

At each time step, the decoder predicts a probability distribution for the output tokens. As with language modeling, we can apply softmax to obtain the distribution and calculate the cross-entropy loss for optimization. Recall from Section 10.5 that the special padding tokens are appended to the end of sequences and so sequences of varying lengths can be efficiently loaded in minibatches of the same shape. However, prediction of padding tokens should be excluded from loss calculations. To this end, we can mask irrelevant entries with zero values so that multiplication of any irrelevant prediction with zero equates to zero.

In [ ]:
@d2l.add_to_class(Seq2Seq)
def loss(self, Y_hat, Y):
    l = super(Seq2Seq, self).loss(Y_hat, Y, averaged=False)
    mask = (Y.reshape(-1) != self.tgt_pad).type(torch.float32)
    return (l * mask).sum() / mask.sum()

In [18]:
package Seq2Seq;

sub add_loss_method {
    my ($self, $Y_hat, $Y) = @_;

    # Assume you have a method 'loss' in the parent class and a 'tgt_pad' attribute
    my $l = $self->loss($Y_hat, $Y, averaged => 0);
    my $mask = ($Y->reshape(-1) != $self->{tgt_pad})->to(dtype => 'torch.float32');
    my $masked_loss = ($l * $mask)->sum / $mask->sum;

    return $masked_loss;
}

# 10.7.6. Training
Now we can create and train an RNN encoder–decoder model for sequence-to-sequence learning on the machine translation dataset.

In [ ]:
data = d2l.MTFraEng(batch_size=128)
embed_size, num_hiddens, num_layers, dropout = 256, 256, 2, 0.2
encoder = Seq2SeqEncoder(
    len(data.src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder(
    len(data.tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
model = Seq2Seq(encoder, decoder, tgt_pad=data.tgt_vocab['<pad>'],
                lr=0.005)
trainer = d2l.Trainer(max_epochs=30, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

In [20]:
my $data = new d21::MTFraEng (batch_size=>128);
($embed_size, $num_hiddens, $num_layers, my $dropout) = (256, 256, 2, 0.2); 
$encoder = new Seq2SeqEncoder(
    $data->{src_vocab)->len (), $embed_size, $num_hiddens, $num_layers, $dropout); 
$decoder = new Seq2SeqDecoder(
    $data->{tgt_vocab}->len (), $embed_size, $num_hiddens, $num_layers, $dropout);
my $model = new Seq2Seq ($encoder, $decoder, tgt_pad->$data->{tgt_vocab)->getitem(""<pad>""), lr=>0.005);
my $trainer = new d2l::Trainer (max_epochs >30, gradient_clip_val>1, num_gpus=>1); 
$trainer->fit ($model, $data);

Error: syntax error at reply input line 4, near "src_vocab)"
syntax error at reply input line 7, near "tgt_vocab)"
BEGIN not safe after errors--compilation aborted at reply input line 13.



# 10.7.7. Prediction
To predict the output sequence at each step, the predicted token from the previous time step is fed into the decoder as an input. One simple strategy is to sample whichever token that has been assigned by the decoder the highest probability when predicting at each step. As in training, at the initial time step the beginning-of-sequence (“<bos>”) token is fed into the decoder. This prediction process is illustrated in Fig. 10.7.3. When the end-of-sequence (“&lt;eos&gt;”) token is predicted, the prediction of the output sequence is complete.
![ Fig. 10.7.3 Predicting the output sequence token by token using an RNN encoder--decoder.](../img/seq2seq-predict.svg)

In the next section, we will introduce 
more sophisticated strategies 
based on beam search (Section 10.8).

In [ ]:
@d2l.add_to_class(d2l.EncoderDecoder)  #@save
def predict_step(self, batch, device, num_steps,
                 save_attention_weights=False):
    batch = [a.to(device) for a in batch]
    src, tgt, src_valid_len, _ = batch
    enc_all_outputs = self.encoder(src, src_valid_len)
    dec_state = self.decoder.init_state(enc_all_outputs, src_valid_len)
    outputs, attention_weights = [tgt[:, (0)].unsqueeze(1), ], []
    for _ in range(num_steps):
        Y, dec_state = self.decoder(outputs[-1], dec_state)
        outputs.append(Y.argmax(2))
        # Save attention weights (to be covered later)
        if save_attention_weights:
            attention_weights.append(self.decoder.attention_weights)
    return torch.cat(outputs[1:], 1), attention_weights

In [ ]:
package d2l::EncoderDecoder;

sub add_predict_step_method {
    my ($self, $batch, $device, $num_steps, $save_attention_weights) = @_;

    # Assume the necessary methods and attributes are available within EncoderDecoder
    $batch = [map { $_->to($device) } @$batch];
    my ($src, $tgt, $src_valid_len, undef) = @$batch;

    my $enc_all_outputs = $self->encoder($src, $src_valid_len);
    my $dec_state = $self->decoder->init_state($enc_all_outputs, $src_valid_len);

    my @outputs = ($tgt->slice(':, (0)')->unsqueeze(1));
    my @attention_weights = ();

    for (1..$num_steps) {
        my ($Y, $new_dec_state) = $self->decoder->forward($outputs[-1], $dec_state);
        push @outputs, $Y->argmax(2);
        
        if ($save_attention_weights) {
            push @attention_weights, $self->decoder->attention_weights;
        }

        $dec_state = $new_dec_state;
    }

    my $final_outputs = torch->cat(\@outputs[1..$#outputs], 1);
    return ($final_outputs, \@attention_weights);
}

1;

# 10.7.8. Evaluation of Predicted Sequences
We can evaluate a predicted sequence
by comparing it with the
target sequence (the ground truth).
But what precisely is the appropriate measure 
for comparing similarity between two sequences?


Bilingual Evaluation Understudy (BLEU),
though originally proposed for evaluating
machine translation results (Papineni et al. 2002),
has been extensively used in measuring
the quality of output sequences for different applications.
In principle, for any $n$-gram (Section 9.3.1.1) in the predicted sequence,
BLEU evaluates whether this $n$-gram appears
in the target sequence.

Denote by $p_n$ the precision of an $n$-gram,
defined as the ratio 
of the number of matched $n$-grams in
the predicted and target sequences
to the number of $n$-grams in the predicted sequence.
To explain, given a target sequence $A$, $B$, $C$, $D$, $E$, $F$,
and a predicted sequence $A$, $B$, $B$, $C$, $D$,
we have $p_1 = 4/5$,  $p_2 = 3/4$, $p_3 = 1/3$, and $p_4 = 0$.
Now let $\textrm{len}_{\textrm{label}}$ and $\textrm{len}_{\textrm{pred}}$
be the numbers of tokens in the target sequence 
and the predicted sequence, respectively.
Then, BLEU is defined as

$$ \exp\left(\min\left(0, 1 - \frac{\textrm{len}_{\textrm{label}}}{\textrm{len}_{\textrm{pred}}}\right)\right) \prod_{n=1}^k p_n^{1/2^n},$$
(10.7.4)

where $k$ is the longest $n$-gram for matching.

Based on the definition of BLEU in (10.7.4),
whenever the predicted sequence is the same as the target sequence, BLEU is 1.
Moreover,
since matching longer $n$-grams is more difficult,
BLEU assigns a greater weight
when a longer $n$-gram has high precision.
Specifically, when $p_n$ is fixed,
$p_n^{1/2^n}$ increases as $n$ grows (the original paper uses $p_n^{1/n}$).
Furthermore,
since
predicting shorter sequences
tends to yield a higher $p_n$ value,
the coefficient before the multiplication term in (10.7.4)
penalizes shorter predicted sequences.
For example, when $k=2$,
given the target sequence $A$, $B$, $C$, $D$, $E$, $F$ and the predicted sequence $A$, $B$,
although $p_1 = p_2 = 1$, the penalty factor $\exp(1-6/2) \approx 0.14$ lowers the BLEU.

We implement the BLEU measure as follows.

In [ ]:
def bleu(pred_seq, label_seq, k):  #@save
    """Compute the BLEU."""
    pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, min(k, len_pred) + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[' '.join(label_tokens[i: i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[' '.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[' '.join(pred_tokens[i: i + n])] -= 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score

In [ ]:
use Math::Complex;

sub bleu {
    my ($pred_seq, $label_seq, $k) = @_;
    
    my @pred_tokens = split(' ', $pred_seq);
    my @label_tokens = split(' ', $label_seq);
    
    my $len_pred = scalar(@pred_tokens);
    my $len_label = scalar(@label_tokens);
    
    my $score = exp(min(0, 1 - $len_label / $len_pred));
    
    for my $n (1..min($k, $len_pred)) {
        my ($num_matches, %label_subs);
        
        for my $i (0..($len_label - $n)) {
            $label_subs{join(' ', @label_tokens[$i..($i + $n - 1)])} += 1;
        }
        
        for my $i (0..($len_pred - $n)) {
            if ($label_subs{join(' ', @pred_tokens[$i..($i + $n - 1)])} > 0) {
                $num_matches += 1;
                $label_subs{join(' ', @pred_tokens[$i..($i + $n - 1)])} -= 1;
            }
        }
        
        $score *= ( $num_matches / ($len_pred - $n + 1) ) ** ( 0.5 ** $n );
    }
    
    return $score;
}

In the end, we use the trained RNN encoder–decoder to translate a few English sentences into French and compute the BLEU of the results.

In [ ]:
engs = ['go .', 'i lost .', 'he\'s calm .', 'i\'m home .']
fras = ['va !', 'j\'ai perdu .', 'il est calme .', 'je suis chez moi .']
preds, _ = model.predict_step(
    data.build(engs, fras), d2l.try_gpu(), data.num_steps)
for en, fr, p in zip(engs, fras, preds):
    translation = []
    for token in data.tgt_vocab.to_tokens(p):
        if token == '<eos>':
            break
        translation.append(token)
    print(f'{en} => {translation}, bleu,'
          f'{bleu(" ".join(translation), fr, k=2):.3f}')

In [ ]:
# Datos de ejemplo
my @engs = ('go .', 'i lost .', 'he\'s calm .', 'i\'m home .');
my @fras = ('va !', 'j\'ai perdu .', 'il est calme .', 'je suis chez moi .');

# Suponiendo que `predict_step`, `data.build` y otras funciones están definidas previamente

# Predicciones
my $num_steps = 10; # Aquí debes colocar el valor adecuado
my @preds; # Aquí debes guardar las predicciones obtenidas

# Loop sobre las frases para obtener las traducciones y calcular el BLEU
for my $i (0..$#engs) {
    my $en = $engs[$i];
    my $fr = $fras[$i];
    my $p = $preds[$i]; # Obtén la predicción correspondiente
    
    my @translation;
    foreach my $token (@$p) {
        last if $token eq '<eos>';
        push @translation, $token;
    }
    
    my $translated = join(' ', @translation);
    my $bleu_score = bleu($translated, $fr, 2);
    
    print "$en => $translated, bleu, $bleu_score\n";
}

# 10.7.9. Summary

Following the design of the encoder–decoder architecture, we can use two RNNs to design a model for sequence-to-sequence learning. In encoder–decoder training, the teacher forcing approach feeds original output sequences (in contrast to predictions) into the decoder. When implementing the encoder and the decoder, we can use multilayer RNNs. We can use masks to filter out irrelevant computations, such as when calculating the loss. For evaluating output sequences, BLEU is a popular measure that matches 
-grams between the predicted sequence and the target sequence.

# 10.7.10. Exercises
1. Can you adjust the hyperparameters to improve the translation results?
1. Rerun the experiment without using masks in the loss calculation. What results do you observe? Why?
1. If the encoder and the decoder differ in the number of layers or the number of hidden units, how can we initialize the hidden state of the decoder?
1. In training, replace teacher forcing with feeding the prediction at the previous time step into the decoder. How does this influence the performance?
1. Rerun the experiment by replacing GRU with LSTM.
1. Are there any other ways to design the output layer of the decoder?

